# Imports

In [21]:
import pandas as pd
from pathlib import Path
import numpy as np
import json
import warnings
import math

import dask.dataframe as ddf

import sklearn.linear_model as sklearn_linear_model
import sklearn.metrics as sklearn_metrics
import sklearn.model_selection as sklearn_model_selection
import sklearn.preprocessing as sklearn_preprocessing
import sklearn.feature_selection as sklearn_feature_selection
import sklearn.ensemble as sklearn_ensemble
import sklearn.decomposition as sklearn_decomposition
from sklearn.impute import SimpleImputer

from scipy import stats

from matplotlib import pyplot as plt

import geopandas as gpd
import dask_geopandas as dgpd

import matplotlib.pyplot as plt
import pyreadstat
from pandas.api.types import is_numeric_dtype

In [22]:
data_path = Path('/home/selker/eop/data/malawi')
out_path = Path('/home/selker/eop/eop/select_predictors')
# for reproducibility
RANDOM_STATE=11

# Load cleaned data

In [23]:
year = 2019

malawi = pd.read_parquet(data_path / f'malawi_cleaned_{year}.parquet')
summary = pd.read_parquet(data_path / f'malawi_summary_{year}.parquet')

# Forward selection

Partition your data into 10 random subsets, in order to perform 10-fold cross-validation.
Initialize model parameters m as empty set {}
For counter c from 1 to 30:
   For each variable v (that is not already included in m)
      For each fold i:
            temporarily hold out data in partition i (10% of observations)
            train a model (linear regression?) to predict consumption from {m+v} on the 90% of observations not in i 
                record the in-sample performance (RMSE and R2) on those 90% of observations, and store it [this is the in-sample cross-val performance for fold i]
                using this trained model (trained on the 90% of observations not in i), calculate performance (RMSE and R2) on the held-out observation in i, store those values [these represent the  out-of-sample cross-val performance for fold i]
      Averaging across the 10 folds from the above loop, calculate the average in-sample and out-of-sample performance that you obtain for the model that includes m and v. Store these values.
   Choose the v* that maximizes the out-of-sample performance (among all of the v's tested in the above loop)
   Add v* to your model, and then iterate, until you've identified the best 30 variables

### Common functions

In [4]:
class Score:

    def __init__(self, is_mse, oos_mse, is_r2, oos_r2):
        self.is_mse = is_mse
        self.oos_mse = oos_mse
        self.is_r2 = is_r2
        self.oos_r2 = oos_r2


    def get(self, key):
        if key == 'is_mse':
            return self.is_mse
        elif key == 'oos_mse':
            return self.oos_mse
        elif key == 'is_r2':
            return self.is_r2
        elif key == 'oos_r2':
            return self.oos_r2
        else:
            return ValueError(f'key {key} not recognized')

    def is_improvement_over(self, other):
        if other is None:
            return True
        else:
            return self.oos_r2 > other.oos_r2 

    def __add__(self, other):
        
        if not isinstance(other, Score):
            raise TypeError()

        return Score(
            is_mse=self.is_mse + other.is_mse,
            oos_mse=self.oos_mse + other.oos_mse,
            is_r2=self.is_r2 + other.is_r2,
            oos_r2=self.oos_r2 + other.oos_r2
        )

    def __truediv__(self, denom):
        return Score(
            is_mse=self.is_mse / denom,
            oos_mse=self.oos_mse / denom,
            is_r2=self.is_r2 / denom,
            oos_r2=self.oos_r2 / denom,
        )


def get_columns_for_covariates(covariates):
    columns = []
    for covariate in covariates:
        columns_to_add = list(summary.loc[covariate].columns)
        columns = columns + columns_to_add
    return columns

def evenly_partition(dataset, n_partitions, random_state=None):
    
    shuffled = dataset.sample(frac=1, random_state=random_state)
    indices = np.linspace(0, len(dataset), n_partitions + 1)
    indices = [round(i) for i in indices]
    indices[-1] = len(dataset)
    
    folds = []
    
    for i in range(n_partitions):
        test = shuffled[indices[i] : indices[i+1]]
        train = pd.concat((
            shuffled[:indices[i]], shuffled[indices[i+1]:]
        ))
        folds.append((train, test))
    
    return folds


def forward_select_features(dataset, candidate_covariates, num_to_select=30, random_state=None):

    n_folds = 10
    
    folds = evenly_partition(dataset, n_folds, random_state)
    
    selected_covariate_list = []
    columns_so_far = []
    
    cumulative_scores = []

    # avoid set-order non-determinism
    unselected_covariates = list(candidate_covariates.copy())
    unselected_covariates.sort()

    global global_values
    global_values = dict()

    while((len(unselected_covariates) > 0) and (len(selected_covariate_list) < num_to_select)):
        
        best_score_this_step = None
        best_covariate_this_step = None

        for covariate in unselected_covariates:

            # the values in the summary are a numpy array - need a list for list concatenation to work.
            columns_to_add = list(summary.loc[covariate].columns)
            columns_to_try = columns_so_far + columns_to_add

            fold_scores = []
            i = 0
            for train, test in folds:

                # fit the model on training data
                lr = sklearn_linear_model.LinearRegression()
                lr.fit(
                    train[columns_to_try], 
                    train.outcome,
                    sample_weight=train.hh_wgt
                )
                
                # Make predictions on training data, score
                y_pred = lr.predict(train[columns_to_try])
                is_mse = sklearn_metrics.mean_squared_error(
                    train.outcome, y_pred, sample_weight=train.hh_wgt
                )
                is_r2 = sklearn_metrics.r2_score(
                    train.outcome, y_pred, sample_weight=train.hh_wgt
                )

                # Make predictions on test data, score
                y_pred = lr.predict(test[columns_to_try])
                oos_mse = sklearn_metrics.mean_squared_error(
                    test.outcome, y_pred, sample_weight=test.hh_wgt
                )
                oos_r2 = sklearn_metrics.r2_score(
                    test.outcome, y_pred, sample_weight=test.hh_wgt
                )

                fold_scores.append(Score(
                    is_mse=is_mse, is_r2=is_r2, oos_mse=oos_mse, oos_r2=oos_r2
                ))

                global_values[(len(selected_covariate_list), covariate, i)] = (
                    test.outcome, y_pred
                )
                i += 1
            average_scores = sum(
                fold_scores, start=Score(0,0,0,0)
            ) / n_folds

            if average_scores.is_improvement_over(best_score_this_step):

                best_score_this_step = average_scores
                best_covariate_this_step = covariate
                best_columns_this_step = columns_to_add
        
        if (
            (len(cumulative_scores) < 1) 
            or (best_score_this_step.is_improvement_over(cumulative_scores[-1]))
        ):
            selected_covariate_list.append(best_covariate_this_step)
            columns_so_far += best_columns_this_step
            cumulative_scores.append(best_score_this_step)
            unselected_covariates.remove(best_covariate_this_step)
    
        else:
            print('No more improvement.')
            break

    selected_covariates = pd.DataFrame(
        selected_covariate_list, 
        columns=['covariate']
    )

    for score_type in ('is_mse', 'oos_mse', 'is_r2', 'oos_r2'):
        selected_covariates[score_type] = [s.get(score_type) for s in cumulative_scores]
        selected_covariates[score_type] = selected_covariates[score_type].astype(float).round(3)
  
    selected_covariates = (
        selected_covariates.join(summary['description'], on='covariate', how='left')
    )
    
    return add_covariate_r2(dataset, selected_covariates)

def add_covariate_r2(dataset, selected_covariates):
    
    r2s_univariate = []

    selected_covariate_list = selected_covariates.covariate.values
    for selected_covariate in selected_covariate_list:
        columns = list(summary.loc[selected_covariate].columns)
        lr = sklearn_linear_model.LinearRegression()
        lr.fit(
            dataset[columns], 
            dataset.outcome,
            sample_weight=dataset.hh_wgt
        )

        y_pred = lr.predict(dataset[columns])
        
        r2 = sklearn_metrics.r2_score(
            dataset.outcome, y_pred, sample_weight=dataset.hh_wgt
        )
        r2s_univariate.append(r2)
    
    selected_covariates['single_covariate_r2'] = r2s_univariate
    selected_covariates['single_covariate_r2'] = (
        selected_covariates['single_covariate_r2'].astype(float).round(3)
    )

    return selected_covariates

In [5]:
if year == 2016:
    manually_excluded = {
        'outcome',
        'hh_wgt',
        'af_bio_1', # annual mean temp
        'hh_f01_4a', # This and next 3: confusing questions about names listed on ownership doc for property
        'hh_f01_4b',
        'hh_f01_4c',
        'hh_f01_4d',
        'asset_index',
    } 
    
    _, consumption_metadata = pyreadstat.read_dta(
            data_path / 'MWI_2016_IHS-IV_v04_M_STATA14/consumption_aggregate/ihs4 consumption aggregate.dta', metadataonly=True
    )


elif year == 2019:
    manually_excluded = {
        'asset_index',
        'hh_a02a', 
        'hh_a03', 
        'hh_a23', 
        'hh_a22', 
        'interviewDate', 
        'consumption_ppp_2017', 
        'hh_f18', # total value of firewood you used in the last week
        'index_mosaiks',
        'ea_id', # debatable
        'hh_f26a'
    }

    durable_verifiable_modules = {
        'hh_mod_a_filt',
        # 'HH_MOD_F',
        'HH_MOD_X',
        'ag_mod_a',
        'householdgeovariables_ihs5',
        'HH_MOD_L_durable_goods',
        'HH_MOD_M_ag_goods'
    }

    non_durable_verifiable_covariates_override = {
        'hh_g09',
        'hh_s01',
        'hh_w01',
        'hh_x09',
        'hh_a06',
        'hh_a11',
        'hh_a13',
        'hh_a22',
        'hh_a23',
        'ag_s01',
    }
    durable_verifiable_covariates_override= {
        'area',
        'district',
        'region',
        'hhsize',
        'urban',
        'num_adults',
        'num_children',
        'hh_f01',
        'yearly_rent',
        'hh_f06',
        'hh_f07',
        'hh_f07_oth',
        'hh_f08',
        'hh_f08_oth',
        'hh_f09',
        'hh_f09_oth',
        'hh_f10',
        'hh_f11',
        'hh_f11_oth',
        'hh_f12',
        'hh_f12_oth',
        'hh_f19',
        'hh_f31',
        'hh_f34',
        'hh_f36',
        'hh_f36_oth',
        'hh_f41',
        'hh_f41_oth',
        'hh_f43',
        'hh_f43_oth',
        'TA',
        'adulteq',
    }
        
    _, consumption_metadata = pyreadstat.read_dta(
        data_path / 'MWI_2019_IHS-V_v06_M_Stata/ihs5_consumption_aggregate.dta', metadataonly=True
    )

consumption_columns_excluded = (
    set(consumption_metadata.column_names)
    # columns we don't want to exclude as consumption columns
    - {'region', 'district', 'ea_id', 'area', 'urban', 'hhsize'} 
)

mosaiks_columns_excluded = {
    covariate for covariate in summary.index if 'mosaiks' in covariate
}

columns_excluded = consumption_columns_excluded | mosaiks_columns_excluded | manually_excluded

covariates_to_consider = set(summary[summary.type != 'dropped'].index.values) - columns_excluded

summary_to_consider = summary[summary.index.isin(covariates_to_consider)]
durable_verifiable_covariates_table = summary_to_consider[
    (
        (summary_to_consider.module.isin(durable_verifiable_modules))
        | (summary_to_consider.index.isin(durable_verifiable_covariates_override))
    )
    & ~(summary_to_consider.index.isin(non_durable_verifiable_covariates_override))
]
non_durable_verifiable_covariates_table = summary_to_consider[~summary_to_consider.index.isin(durable_verifiable_covariates_table.index)]

durable_verifiable_covariates = set(durable_verifiable_covariates_table.index.values)

In [6]:
durable_verifiable_covariates_table.reset_index()[['covariate']].to_csv(
    out_path / f'{year}' / 'full_set_durable_verifiable_covariates.csv', index=False
)

In [7]:
summary[summary.index.str.contains('nearest_mosaiks')].reset_index()[['covariate']].to_csv(
    out_path / f'{year}' / 'mosaiks_nearest_features.csv', index=False
)

### Using forward selection from the full set

All cov, all households

In [8]:
%%time
selected_covariates = forward_select_features(
    malawi, covariates_to_consider, num_to_select = 30, random_state=RANDOM_STATE
)
selected_covariates.to_csv(out_path / f'{year}' / 'covariates_country_all.csv', index=False)

No more improvement.
CPU times: user 5h 23min 18s, sys: 23h 14min 22s, total: 1d 4h 37min 40s
Wall time: 37min 29s


Durable/verifiable cov, all households

In [ ]:
%%time
selected_covariates_durable = forward_select_features(
    malawi, durable_verifiable_covariates, num_to_select = 30, random_state=RANDOM_STATE
)
selected_covariates_durable.to_csv(out_path / f'{year}' / 'covariates_country_durable.csv', index=False)

Durable/verifiable cov, no geo fixed effects, all households

In [19]:
durable_verifiable_no_geo = durable_verifiable_covariates - {
    'district', 'ea_id', 'region', 'area', 'ea_lat_mod', 'ea_lon_mod', 
}

In [20]:
durable_verifiable_no_geo

{'af_bio_12_x',
 'af_bio_13_x',
 'af_bio_16_x',
 'af_bio_1_x',
 'af_bio_8_x',
 'afmnslp_pct',
 'ag_asset_AXE',
 'ag_asset_BARN',
 'ag_asset_CHICKEN HOUSE',
 'ag_asset_FORK',
 'ag_asset_GENERATOR',
 'ag_asset_GRAIN MILL',
 'ag_asset_GRANARY',
 'ag_asset_GRASS CUTTER',
 'ag_asset_HAMMER',
 'ag_asset_HAND HOE',
 'ag_asset_HARLOW',
 'ag_asset_INCUBATOR',
 'ag_asset_LIVESTOCK KRAAL',
 'ag_asset_MOTORISED PUMP',
 'ag_asset_MPHOPO',
 'ag_asset_OX CART',
 'ag_asset_OX PLOUGH',
 'ag_asset_PANGA KNIFE',
 'ag_asset_PICK AXE',
 'ag_asset_PIG STY',
 'ag_asset_POULTRY KRAAL',
 'ag_asset_RIDGER',
 'ag_asset_SHOVEL',
 'ag_asset_SICKLE',
 'ag_asset_SLASHER',
 'ag_asset_SPRAYER',
 'ag_asset_STORAGE HOUSE',
 'ag_asset_TREADLE PUMP',
 'ag_asset_WATERING CAN',
 'ag_asset_WHEEL BARROW',
 'ag_b101',
 'ag_i101b',
 'anntot_avg',
 'cropshare',
 'dist_admarc',
 'dist_agmrkt',
 'dist_auction',
 'dist_boma',
 'dist_border',
 'dist_popcenter',
 'dist_road',
 'durable_asset_Air conditioner',
 'durable_asset_Bed',
 '

In [ ]:
%%time
selected_covariates_durable = forward_select_features(
    malawi, durable_verifiable_covariates, num_to_select = 30, random_state=RANDOM_STATE
)
selected_covariates_durable.to_csv(out_path / f'{year}' / 'covariates_country_durable.csv', index=False)

### Using forward selection on a subset of households

In [16]:
malawi_rural = malawi[malawi.reside_URBAN == 0]

In [17]:
%%time
selected_covariates_rural = forward_select_features(
    malawi_rural, covariates_to_consider, num_to_select = 30, random_state=RANDOM_STATE
)
selected_covariates_rural.to_csv(out_path / f'{year}' / 'covariates_rural_all.csv', index=False)


CPU times: user 14h 12min 15s, sys: 2d 10h 21min, total: 3d 33min 15s
Wall time: 1h 35min 2s


In [23]:
%%time
durable_verifiable_selected_covariates_rural = forward_select_features(
    malawi_rural, durable_verifiable_covariates, num_to_select = 30, random_state=RANDOM_STATE
)
durable_verifiable_selected_covariates_rural.to_csv(
    out_path / f'{year}' / 'covariates_rural_durable.csv', index=False
)

CPU times: user 4h 18min 29s, sys: 18h 20min 51s, total: 22h 39min 21s
Wall time: 29min 38s


#### Only households < $5/person/day

In [8]:
malawi_below_5 = malawi[malawi.outcome < 5]

In [ ]:
%%time
selected_covariates_below_5 = forward_select_features(
    malawi_below_5, covariates_to_consider, num_to_select = 30, random_state=RANDOM_STATE
)
selected_covariates_below_5.to_csv(out_path / f'{year}' / 'covariates_below_5_all.csv', index=False)


In [9]:
%%time
durable_verifiable_selected_covariates_below_5 = forward_select_features(
    malawi_below_5, durable_verifiable_covariates, num_to_select = 30, random_state=RANDOM_STATE
)
durable_verifiable_selected_covariates_below_5.to_csv(
    out_path / f'{year}' /  'covariates_below_5_durable.csv', index=False
)


CPU times: user 5h 16min 20s, sys: 22h 55min 33s, total: 1d 4h 11min 54s
Wall time: 37min 43s


#### By district

In [30]:
%%time
malawi_below_5 = malawi[malawi.outcome < 5]
district_columns = [c for c in malawi.columns if c.startswith('district')]
for c in district_columns:
    district = c.split('_')[1]
    in_district = malawi_below_5[malawi_below_5[c] == 1]
    if len(in_district) < 100:
        print(f'data size for {district} is small: {len(in_district)}')

    selected_covariates = forward_select_features(
        in_district, durable_verifiable_covariates, num_to_select = 10, random_state=RANDOM_STATE
    )

    selected_covariates.to_csv(Path('district_level_covariates') / f'{district}.csv', index=False)

data size for Likoma is small: 28
No more improvement.
CPU times: user 1h 28min 59s, sys: 3h 37min 14s, total: 5h 6min 13s
Wall time: 47min 48s


### Print output tables

In [ ]:
selected_covariates_durable = pd.read_csv(out_path / f'{year}' /  'covariates_country_durable.csv')
selected_covariates = pd.read_csv(out_path / f'{year}' /  'covariates_country_all.csv')
selected_covariates_rural = pd.read_csv(out_path / f'{year}' /  'covariates_rural_all.csv')
selected_covariates_rural_durable = pd.read_csv(out_path / f'{year}' /  'covariates_rural_durable.csv')
selected_covariates_under_5 = pd.read_csv(out_path / f'{year}' /  'covariates_below_5_all.csv')

In [ ]:
selected_covariates_under_5_durable = pd.read_csv(out_path / f'{year}' /  'covariates_below_5_durable.csv')

with pd.option_context('display.max_rows', 200, 'display.max_colwidth', None):

    display(
        selected_covariates_under_5_durable[['covariate', 'description', 'oos_r2', 'is_r2', 'single_covariate_r2']]
        .set_index('covariate')
    )

### Output column list

In [168]:
selected_covariates[~selected_covariates.Covariate.isna()].Covariate.to_csv('2016/selected_columns_no_mosaiks.csv', index=False)

In [ ]:
columns = pd.read_csv('2016/selected_columns_no_mosaiks.csv')

##### Shuffle to simulate less-selected features

In [24]:
columns_shuffled = columns.sample(frac=1)

In [26]:
columns_shuffled.to_csv('selected_columns_shuffled.csv', index=False)

# Stand-alone models

In [423]:
def fit_and_evaluate_models(
    ds, columns, groups=None, outcome='outcome', lasso_alphas=[3e-6, 1e-5, 3e-5], gb=False,
    n_pca_components=None, linear=True
):

    if groups:
        groups = ds[groups]
        cv = sklearn_model_selection.GroupKFold(n_splits=5)
    else:
        cv = sklearn_model_selection.KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


    if n_pca_components:
        pca = sklearn_decomposition.PCA(n_components=n_pca_components)
        pca_predictors = pca.fit_transform(ds[columns])
        pca_predictors = pd.DataFrame(data=pca_predictors)
        pca_columns = [f'pca_{c}' for c in pca_predictors.columns]
        pca_predictors.columns = pca_columns
        
        pca_predictors[['hh_wgt', outcome]] = (
            ds[['hh_wgt', outcome]]
        )

        columns = pca_columns
        ds = pca_predictors

    results = dict()

    if linear:
        lr = sklearn_linear_model.LinearRegression()
        cross_val = sklearn_model_selection.cross_validate(
            lr,
            X=ds[columns], 
            y=ds[outcome],
            groups=groups,
            params={'sample_weight': ds.hh_wgt},
            scoring='r2',
            cv=cv,
            return_train_score=True
        )
    
        results['lr_oos'] = cross_val['test_score'].mean()
        results['lr_is'] = cross_val['train_score'].mean()
    
        lasso = sklearn_linear_model.Lasso()
    
        lasso_grid_search = sklearn_model_selection.GridSearchCV(
            lasso,
            {'alpha': lasso_alphas},
            scoring='r2',
            cv=cv,
            n_jobs=40,
            return_train_score=True
        
        )
        lasso_grid_search.fit(    
            X=ds[columns], 
            y=ds[outcome],
            sample_weight=ds.hh_wgt,
            groups=groups
        )
    
        results['lasso_best_alpha'] = lasso_grid_search.best_params_['alpha']
        results['lasso_oos'] = lasso_grid_search.best_score_

    if gb:
        
        """
        class sklearn.ensemble.GradientBoostingRegressor(*, loss='squared_error', learning_rate=0.1, n_estimators=100,
        subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
        max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, 
        max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0
        )
        
        Parameter grid from chi et al:
        Hyperparameters were tuned to minimize the cross-validated mean-squared error, using a grid search over 
        several possible values for maximum tree depth (1, 3, 5, 10, 15, 20, 31) and the minimum sum of instance 
        weight needed in a child (1, 3, 5, 7, 10).
        """
        hyperparameters = {
            'min_samples_leaf': [1,7,20],
            'max_depth': [1, 5, 10, 25],
        }
        gb_classifier = sklearn_ensemble.GradientBoostingRegressor()
        gb_grid_search = sklearn_model_selection.GridSearchCV(
            gb_classifier, 
            hyperparameters, 
            scoring='r2',
            cv=cv,
            verbose=1,
            n_jobs=40
        )
        
        gb_grid_search.fit(    
            X=ds[columns], 
            y=ds[outcome],
            sample_weight=ds.hh_wgt,
            groups=groups
        )

        results['gb_oos'] = gb_grid_search.best_score_
        

    return results

## Geo-only models

In [393]:
district_columns = [c for c in malawi.columns if 'district' in c]
ea_columns = [c for c in malawi.columns if 'ea_id' in c]

In [439]:
malawi['ea_id'] = None

for c in ea_columns:
    ea_id = c.split('_')[2]
    malawi.loc[malawi[c] == 1, 'ea_id'] = int(ea_id)


In [500]:
malawi['log_consumption'] = np.log(malawi.outcome)

0       -0.184338
1       -0.208366
2       -0.609822
3        0.934817
4        0.408347
           ...   
11424   -0.946083
11425    0.293111
11426    2.000725
11427    1.512548
11428    0.133752
Name: log_asset_index, Length: 11429, dtype: float64

In [501]:
all_results_geo = []
for column_set_name, columns in zip(
    ('district', 'ea'),
    (district_columns, ea_columns)
):
    for outcome in ('log_consumption',):
       
        lasso_alphas=[3e-5, 1e-4, 3e-4]
        results = fit_and_evaluate_models(
            malawi, 
            columns=columns,
            outcome=outcome,
            lasso_alphas=lasso_alphas,
            gb=True
        )

        results.update({
            'columns': column_set_name,
            'outcome': outcome,
        })

        all_results_geo.append(results)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [502]:
summary_table = pd.DataFrame.from_dict(all_results_geo).drop_duplicates(
    subset=['columns', 'outcome', ], keep='last'
)[['outcome', 'lr_oos', 'lr_is', 'lasso_oos', 'lasso_best_alpha', 'gb_oos', 'columns']]

In [503]:
summary_table

,outcome,lr_oos,lr_is,lasso_oos,lasso_best_alpha,gb_oos,columns
0,log_consumption,0.196331,0.202703,0.196284,0.00003,0.196250,district
1,log_consumption,0.217010,0.340081,0.247126,0.00003,0.203808,ea


In [440]:
more_results = []

for column_set_name, columns in zip(
    ('ea_lat_lon', 'ea_id'),
    (['ea_lat_mod', 'ea_lon_mod'], ['ea_id'])
):
    for outcome in ('outcome', 'asset_index'):

        lasso_alphas=[1,]
        results = fit_and_evaluate_models(
            malawi, 
            columns=columns,
            outcome=outcome,
            lasso_alphas=lasso_alphas,
            gb=True,
        )
    
        results.update({
            'outcome': outcome,
            'columns': column_set_name
        })
        more_results.append(results)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [441]:
summary_table = pd.DataFrame.from_dict(more_results).drop_duplicates(
    subset=['columns', 'outcome', ], keep='last'
)[['outcome', 'lr_oos', 'lr_is', 'lasso_oos', 'lasso_best_alpha', 'gb_oos', 'columns']]

In [442]:
summary_table

,outcome,lr_oos,lr_is,lasso_oos,lasso_best_alpha,gb_oos,columns
0,outcome,0.001373,0.001976,-0.004751,1,0.188309,ea_lat_lon
1,asset_index,0.004407,0.005076,-0.004039,1,0.199922,ea_lat_lon
2,outcome,-0.000481,-0.000023,-0.000481,1,0.187978,ea_id
3,asset_index,0.001571,0.002079,0.001571,1,0.197413,ea_id


## Obtain LASSO coefficients to check fixed effects

In [460]:
lasso = sklearn_linear_model.Lasso()

lasso_grid_search = sklearn_model_selection.GridSearchCV(
    lasso,
    {'alpha': [1e-4,]},
    scoring='r2',
    cv=sklearn_model_selection.KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    n_jobs=40,
    refit=True,
)

lasso_grid_search.fit(    
    X=malawi[ea_columns],
    y=malawi['outcome'],
    sample_weight=malawi.hh_wgt,
)

GridSearchCV(cv=KFold(n_splits=5, random_state=11, shuffle=True),
             estimator=Lasso(), n_jobs=40, param_grid={'alpha': [0.0001]},
             scoring='r2')

In [454]:
lasso_grid_search.best_estimator_

Lasso(alpha=0.0001)

In [482]:
fixed_effects = pd.DataFrame(
    data=np.array([ea_columns,  lasso_grid_search.best_estimator_.coef_]).transpose(),
    columns=['ea_id', 'coefficient']
)

fixed_effects.ea_id = fixed_effects.ea_id.apply(lambda c: int(c.split('_')[2]))


In [483]:
ea_locations = malawi.groupby('ea_id')[['ea_lat_mod', 'ea_lon_mod']].first()
ea_locations_with_fixed_effects = fixed_effects.join(ea_locations, on='ea_id')

In [485]:
ea_locations_with_fixed_effects.to_csv('ea_locations_with_lasso_fixed_effects.csv', index=False)

## Mosaiks models

### Prep datasets 

In [294]:
mosaiks_columns_nearest = [c for c in malawi.columns if c.startswith('nearest_mosaiks')]
mosaiks_columns_voronoi = [c for c in malawi.columns if c.startswith('voronoi_mosaiks')]
mosaiks_columns_admin_3 = [c for c in malawi.columns if c.startswith('admin_3_mosaiks')]

In [ ]:
mosaiks_columns = mosaiks_columns_nearest + mosaiks_columns_voronoi + mosaiks_columns_admin_3

mosaiks_means = malawi[mosaiks_columns].mean()
mosaiks_stds = malawi[mosaiks_columns].std()
malawi_mosaiks_normalized = (malawi[mosaiks_columns] - mosaiks_means) / mosaiks_stds
# std of 0 -> div by 0 in previous step; fill with zeros.
malawi_mosaiks_normalized.fillna(value=0, inplace=True)

malawi_mosaiks_normalized[['case_id', 'hh_wgt', 'outcome', 'asset_index']] = (
    malawi[['case_id', 'hh_wgt', 'outcome', 'asset_index']]
)

ea_columns = [c for c in malawi.columns if 'ea_id' in c]
malawi_mosaiks_normalized['ea_id'] = None

for c in ea_columns:
    ea_id = c.split('_')[2]
    malawi_mosaiks_normalized.loc[malawi[c] == 1, 'ea_id'] = str(ea_id)

malawi_mosaiks_normalized = malawi_mosaiks_normalized.sample(frac=1, random_state=RANDOM_STATE)

mosaiks_by_ea = malawi_mosaiks_normalized[['ea_id'] + mosaiks_columns].groupby('ea_id').first()

wgt_by_ea = malawi_mosaiks_normalized[['ea_id', 'hh_wgt']].groupby('ea_id').sum()

mosaiks_by_ea = mosaiks_by_ea.join(wgt_by_ea)
mosaiks_by_ea = mosaiks_by_ea.sample(frac=1, random_state=RANDOM_STATE)

In [ ]:
pca = sklearn_decomposition.PCA(n_components=200)
mosaiks_pca_array = pca.fit_transform(malawi_mosaiks_normalized[mosaiks_columns_nearest])
mosaiks_pca = pd.DataFrame(data=mosaiks_pca_array)
mosaiks_pca_columns = [f'pca_{c}' for c in mosaiks_pca.columns]
mosaiks_pca.columns = mosaiks_pca_columns

mosaiks_pca[['case_id', 'hh_wgt', 'outcome', 'asset_index', 'ea_id']] = (
    malawi_mosaiks_normalized[['case_id', 'hh_wgt', 'outcome','asset_index', 'ea_id']]
)

### Fit models

In [408]:
for column_set_name, columns in zip(
    ('nearest',),
    (mosaiks_columns_nearest,)
):
    for groups in ('ea_id', None):
        for outcome in ('outcome', 'asset_index'):
            for pca in (None, 100):
                if groups:
                    lasso_alphas=[3e-2, 1e-1, 3e-1]
                else:
                    lasso_alphas=[3e-5, 1e-4, 3e-4]

                results = fit_and_evaluate_models(
                    malawi_mosaiks_normalized, 
                    columns=columns,
                    groups=groups,
                    outcome=outcome,
                    lasso_alphas=lasso_alphas,
                    n_pca_components=pca,
                    gb=True
                )

                results.update({
                    'columns': column_set_name,
                    'xval grouped by': groups,
                    'outcome': outcome,
                    'pca': pca is not None,
                    'n_pca': pca
                })
    
                all_results_mosaiks.append(results)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.753e+03, tolerance: 4.955e+00
  model = cd_fast.enet_coordinate_descent(
/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.130e+04, tolerance: 5.315e+00
  model = cd_fast.enet_coordinate_descent(
/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.096e+04, tolerance: 4.823e+00
  model = cd_fast.enet_coordinate_descent(
/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.162e+04, tolerance: 5.149e+00
  model = cd_fast.enet_coordinate_descent(
/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/selker/.conda/envs/leo_base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.196e+04, tolerance: 6.436e+00
  model = cd_fast.enet_coordinate_descent(


Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [411]:
summary_table = pd.DataFrame.from_dict(all_results_mosaiks).drop_duplicates(
    subset=['columns', 'xval grouped by', 'outcome', 'pca', 'n_pca'], keep='last'
)[['outcome', 'xval grouped by', 'n_pca', 'lr_oos', 'lr_is', 'lasso_oos', 'lasso_best_alpha', 'gb_oos']]

In [413]:
summary_table[summary_table.n_pca.isin((np.nan, 100))]

,outcome,xval grouped by,n_pca,lr_oos,lr_is,lasso_oos,lasso_best_alpha,gb_oos
22,outcome,ea_id,NaN,-5.644263e+20,0.275798,0.086886,0.03000,0.082417
23,outcome,ea_id,100.0,-1.700986e+00,0.148022,0.096654,0.03000,0.068117
24,asset_index,ea_id,NaN,-3.567169e+20,0.289640,0.090313,0.03000,0.084398
25,asset_index,ea_id,100.0,-1.041065e+00,0.158484,0.091039,0.03000,0.064348
26,outcome,None,NaN,1.682261e-01,0.281007,0.172431,0.00003,0.181145
27,outcome,None,100.0,1.228690e-01,0.143696,0.123472,0.00030,0.179030
28,asset_index,None,NaN,1.819511e-01,0.295989,0.181113,0.00003,0.189553
29,asset_index,None,100.0,1.315020e-01,0.153767,0.132111,0.00030,0.188197


## Univariate r2s for all covariates considered

In [214]:
r2s_univariate = []
malawi_covariate_columns_list = list(covariates_to_consider)
for selected_covariate in malawi_covariate_columns_list:
    lr = sklearn_linear_model.LinearRegression()
    lr.fit(
        malawi[[selected_covariate]], 
        malawi.outcome,
        sample_weight=malawi.hh_wgt
    )
    # Make predictions on test data
    y_pred = lr.predict(malawi[[selected_covariate]])
    
    r2 = sklearn_metrics.r2_score(
        malawi.outcome, y_pred, sample_weight=malawi.hh_wgt
    )
    r2s_univariate.append(r2)

In [ ]:
all_univariate_r2s = pd.DataFrame(
    data=np.array([malawi_covariate_columns_list, r2s_univariate]).transpose(),
    columns=['covariate', 'univariate_r2']
)
all_univariate_r2s.univariate_r2 = all_univariate_r2s.univariate_r2.astype(float)
display(all_univariate_r2s.sort_values('univariate_r2', ascending=False).head(50))

## Print summaries of covariates

In [ ]:
# We determine what is included by omitting what's not included. This approach handles one-hot encoded
# columns correctly.

covariates_considered = [c for c in not_dropped_for_missingness if c not in columns_excluded]
with pd.option_context('display.max_rows', 300, 'display.max_colwidth', 1):

    display(
        summary[
            summary.covariate.isin(covariates_considered)
        ]
        [['covariate', 'description', 'missing_fraction', 'mean', 'median', 'std']]
    )

In [ ]:
with pd.option_context('display.max_rows', 300, 'display.max_colwidth', 1):

    display(
        summary[
            (summary.covariate.isin(columns_excluded))
            & (~summary.covariate.isin(dropped_for_missingness))
        ]
        [['covariate', 'description', 'missing_fraction', 'mean', 'median', 'std']]
    )

In [ ]:
with pd.option_context('display.max_rows', 200, 'display.max_colwidth', 1):

    display(
        summary[summary.covariate.isin(dropped_for_missingness)]
        [['covariate', 'description', 'missing_fraction', 'mean', 'median', 'std']]
    )

# Manually construct + write covariate sets

In [9]:
summary_r = summary.reset_index()

In [19]:
summary_r[summary_r.covariate.str.startswith('district')]['columns'].values

array([array(['district_Balaka', 'district_Blantyre', 'district_Blantyre City',
              'district_Chikwawa', 'district_Chiradzulu', 'district_Chitipa',
              'district_Dedza', 'district_Dowa', 'district_Karonga',
              'district_Kasungu', 'district_Likoma', 'district_Lilongwe',
              'district_Lilongwe City', 'district_Machinga', 'district_Mangochi',
              'district_Mchinji', 'district_Mulanje', 'district_Mwanza',
              'district_Mzimba', 'district_Mzuzu City', 'district_Neno',
              'district_Nkhatabay', 'district_Nkhotakota', 'district_Nsanje',
              'district_Ntcheu', 'district_Ntchisi', 'district_Phalombe',
              'district_Rumphi', 'district_Salima', 'district_Thyolo',
              'district_Zomba City', 'district_Zomba Non-City'], dtype=object)   ],
      dtype=object)